In [10]:
import pandas as pd 
from dash_website import MAIN_CATEGORIES_TO_CATEGORIES
from dash_website.xwas.univariate_results_tabs import ITEMS_LEGEND
import plotly.graph_objects as go


summary = pd.read_feather("../../data/xwas/univariate_results/summary.feather").set_index(["dimension", "category"])
summary.columns = pd.MultiIndex.from_tuples(
    list(map(eval, summary.columns.tolist())), names=["category", "variable"]
)
item = "significant"
main_category = list(MAIN_CATEGORIES_TO_CATEGORIES.keys())[-1]

In [11]:
summary_item_percentage = (
    100
    * summary.reset_index().pivot(
        index=[("dimension", "")], columns=[("category", "")], values=(item, "percentage")
    )
).astype(int)
summary_item_percentage_category = summary_item_percentage[MAIN_CATEGORIES_TO_CATEGORIES[main_category]]
summary_item_percentage_category.index.name = "dimension"
summary_item_percentage_category.columns.name = "category"

summary_item_number = summary.reset_index().pivot(
    index=[("dimension", "")], columns=[("category", "")], values=(item, "number")
)
summary_item_number_category = summary_item_number[MAIN_CATEGORIES_TO_CATEGORIES[main_category]]
summary_item_percentage_category.index.name = "dimension"
summary_item_percentage_category.columns.name = "category"

hovertemplate = "<br>".join(
    [
        "X main category: %{x}",
        "Aging dimension: %{y}",
        f"{ITEMS_LEGEND[item]}: " + "%{customdata} ~ %{z} % of the variables",
        "<extra></extra>",
    ]
)

heatmap = go.Heatmap(
    z=summary_item_percentage_category,
    x=summary_item_percentage_category.columns,
    y=summary_item_percentage_category.index,
    customdata=summary_item_number_category,
    hovertemplate=hovertemplate,
)

fig = go.Figure(heatmap)
fig.update_layout(
    {
        "title": f"Percentage of {ITEMS_LEGEND[item]}",
        "xaxis": {"title": "X subcategory", "tickangle": 90},
        "yaxis": {"title": "Aging dimension"},
        "width": max(30 * summary_item_percentage_category.shape[1], 500),
        "height": 30 * summary_item_percentage_category.shape[0],
    }
)